In [1]:
import hls4ml
import torch
import torch.nn as nn
import numpy as np

2024-09-12 11:37:34.160427: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-12 11:37:34.160472: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-12 11:37:34.160481: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-09-12 11:37:34.240039: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
class LayerNorm(nn.Module):
    def __init__(self):
        super().__init__()
        self.layernorm = nn.LayerNorm(5)
        
    def forward(self, x):
        return self.layernorm(x)

In [5]:
X_input = np.array([[[0.90111183, 0.89834848, 0.7941329, 0.36237572, 0.77297575],
 [0.8461442, 0.80346333, 0.91561476, 0.03342599, 0.91221495],
 [0.94028454, 0.09336006, 0.18070355, 0.95906708, 0.80373106],
 [0.21973088, 0.47594852, 0.71696623, 0.70568395, 0.56423564]]])
# X_input = np.random.rand(1, 4, 5)
print(X_input[0])

[[0.90111183 0.89834848 0.7941329  0.36237572 0.77297575]
 [0.8461442  0.80346333 0.91561476 0.03342599 0.91221495]
 [0.94028454 0.09336006 0.18070355 0.95906708 0.80373106]
 [0.21973088 0.47594852 0.71696623 0.70568395 0.56423564]]


In [7]:
model = LayerNorm()
model.eval()

pytorch_prediction = model(torch.Tensor(X_input)).detach().numpy()

config = hls4ml.utils.config_from_pytorch_model(model, (None, 4, 5), channels_last_conversion="off", transpose_outputs=False)
output_dir = 'layernorm_hls4ml_test'

hls_model = hls4ml.converters.convert_from_pytorch_model(model, hls_config=config, output_dir=output_dir, backend='Vivado')

hls_model.compile()
hls_prediction = hls_model.predict(X_input)

Interpreting Model ...
Topology:
Layer name: layernorm, layer type: LayerNorm, input shape: [[None, 4, 5]]
Creating HLS model
Writing HLS project
Done
data:
0.900391 0.897461 0.793945 0.362305 0.772461 0.845703 0.802734 0.915039 0.0332031 0.912109 0.939453 0.0927734 0.180664 0.958984 0.803711 0.219727 0.475586 0.716797 0.705078 0.563477 
data:
0.900391 0.772461 0.0332031 0.180664 0.475586 
mean: 0.464844
var: 0.105469
data_diff:
0.433594 0.304688 -0.433594 -0.285156 0.0078125 
deno_inver: 3.0792
res:
1.33496 0.9375 -1.33594 -0.878906 0.0234375 
data:
0.897461 0.845703 0.912109 0.958984 0.716797 
mean: 0.859375
var: 0.00390625
data_diff:
0.0351563 -0.015625 0.0507813 0.0976563 -0.144531 
deno_inver: 16
res:
0.5625 -0.25 0.8125 1.5625 -2.3125 
data:
0.793945 0.802734 0.939453 0.803711 0.705078 
mean: 0.800781
var: 0.00390625
data_diff:
-0.0078125 0 0.136719 0 -0.0976563 
deno_inver: 16
res:
-0.125 0 2.1875 0 -1.5625 
data:
0.362305 0.915039 0.0927734 0.219727 0.563477 
mean: 0.425781
var

In [5]:
print(X_input[0])
print(pytorch_prediction[0])
print(hls_prediction.reshape(4, 5))

[[0.90111183 0.89834848 0.7941329  0.36237572 0.77297575]
 [0.8461442  0.80346333 0.91561476 0.03342599 0.91221495]
 [0.94028454 0.09336006 0.18070355 0.95906708 0.80373106]
 [0.21973088 0.47594852 0.71696623 0.70568395 0.56423564]]
[[ 0.7814519   0.76754916  0.24322537 -1.929007    0.13678074]
 [ 0.4271814   0.300542    0.63330925 -1.9842541   0.62322146]
 [ 0.9096152  -1.3242939  -1.0939103   0.9591574   0.5494319 ]
 [-1.7384648  -0.33237123  0.99030685  0.92839104  0.15213855]]
[[ 0.85351562  0.83300781  0.29101562 -2.02148438  0.16601562]
 [ 0.44824219  0.31835938  0.66113281 -2.00878906  0.64941406]
 [ 0.92675781 -1.33398438 -1.09375     0.97851562  0.56152344]
 [-1.890625   -0.35449219  1.109375    1.0390625   0.18847656]]


In [6]:
diff = pytorch_prediction[0] - hls_prediction.reshape(4, 5)
print(diff)

[[-0.07206374 -0.06545866 -0.04779026  0.09247732 -0.02923489]
 [-0.02106079 -0.01781738 -0.02782357  0.02453494 -0.02619261]
 [-0.01714259  0.00969052 -0.00016034 -0.01935822 -0.01209152]
 [ 0.15216017  0.02212095 -0.11906815 -0.11067146 -0.03633802]]


In [ ]:
model = nn.Sequential()
model.append(nn.Linear(20, 20))
model.append(nn.LayerNorm([20, 20]))
model.append(nn.Tanh())
config = hls4ml.utils.config_from_pytorch_model(model, (None, 20, 20))
hls_model = hls4ml.converters.convert_from_pytorch_model(model, hls_config=config)

In [ ]:
import torch
import numpy as np
input = torch.rand(1, 20, 20)
model_output = model(input)

hls_model.compile()
hls_input = np.ascontiguousarray(input)
hls_output = hls_model.predict(hls_input)

In [ ]:
diff = np.ascontiguousarray(model_output.detach()).flatten() - hls_output
print(diff)